In [ ]:
1. ### Import, Filter, Format Dataset ###

# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns

# TODO: Replace "LOCATION/FILE.csv" with your actual file path
# Example: "data/customer_orders.csv" or "C:/Users/YourName/Documents/orders.csv"
# The CSV should have columns: cust_id, date, ordnum (order number), rev (revenue)
# Delimiter is semicolon (;) and decimal separator is comma (,) - adjust if needed
data = pd.read_csv("LOCATION/FILE.csv", delimiter=";", decimal=",")
data.columns = ["cust_id", "date", "ordnum", "rev"]

# Filter rows where revenue is greater than 0
data = data[data['rev'] > 0]

# Convert date column to datetime object (format inferred automatically)
data['date'] = pd.to_datetime(data['date'])

# K-Means RFM Clustering Analysis

This notebook performs customer segmentation using K-Means clustering on RFM (Recency, Frequency, Monetary) metrics.

## Required Input Data Format

Your CSV file should contain **one row per order** with the following columns:

| Column | Description | Example |
|--------|-------------|---------|
| `cust_id` | Customer ID (unique identifier) | 1001, 1002, etc. |
| `date` | Order date | 2024-01-15, 2024-02-20, etc. |
| `ordnum` | Order number (unique per order) | ORD001, ORD002, etc. |
| `rev` | Revenue amount | 150.50, 299.99, etc. |

### Example CSV Format (semicolon-delimited, comma as decimal separator):
```
cust_id;date;ordnum;rev
1001;2024-01-15;ORD001;150,50
1001;2024-03-20;ORD002;75,25
1002;2024-02-10;ORD003;299,99
1003;2024-01-05;ORD004;50,00
```

**Recommended:** Use 6-24 months of historical order data for best results.

---

In [ ]:
2. ### Creating RFM dataset ###

# RFM stands for Recency, Frequency, Monetary
# - Recency: How many days since the customer's last purchase
# - Frequency: How many unique orders the customer has made
# - Monetary: Total revenue generated by the customer

# Calculate Recency (days since last purchase)
max_date = data['date'].max()
r = data.groupby('cust_id')['date'].max().reset_index()
r['recency'] = (max_date - r['date']).dt.days

# Calculate Frequency (number of unique orders)
f = data.groupby('cust_id').agg(frequency=('ordnum', 'nunique')).reset_index()

# Calculate Monetary (total revenue)
m = data.groupby('cust_id')['rev'].sum().reset_index()
m.rename(columns={'rev': 'monetary'}, inplace=True)

# Merge to create data_rfm
data_rfm = pd.merge(pd.merge(f, r, on='cust_id'), m, on='cust_id')

In [ ]:
3. ### Determining amount of Clusters ###

# The elbow method helps determine the optimal number of clusters
# Look for the "elbow" point where adding more clusters doesn't significantly reduce WSS

# Selecting columns for clustering
data_rfm_elbow = data_rfm[['recency', 'frequency', 'monetary']]

# Function to calculate within-cluster sum of squares (WSS)
def calculate_wss(data, k_max):
    wss = []
    for k in range(1, k_max + 1):
        kmeans = KMeans(n_clusters=k, random_state=1234).fit(data)
        wss.append(kmeans.inertia_)
    return wss

# Using the WSS function to find the optimal number of clusters
# You can change k_max (currently 5) to test more cluster options
wss = calculate_wss(data_rfm_elbow, 5)

# Plotting the elbow plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), wss, marker='o', linestyle='-', color='b')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters')
plt.ylabel('Within groups sum of squares')
plt.show()

In [ ]:
4. ### Performing RFM Analysis ###

# TODO: Insert the optimal number of clusters based on the elbow plot above
# Replace the number in n_clusters below (currently set to 5)
kmeans_basic = KMeans(n_clusters=5, random_state=1234).fit(data_rfm_elbow)
cluster_labels = kmeans_basic.labels_

# Creating a DataFrame with cluster assignments and centers
kmeans_basic_df = pd.DataFrame({'Cluster': cluster_labels})
kmeans_basic_centers = pd.DataFrame(kmeans_basic.cluster_centers_, columns=['Center_Recency', 'Center_Frequency', 'Center_Monetary'])

# Count the number of instances in each cluster
cluster_sizes = pd.Series(cluster_labels).value_counts().sort_index()

# Add the cluster size information to the centers DataFrame
kmeans_basic_centers['GroupSize'] = cluster_sizes.values

# Add cluster label to the centers DataFrame
kmeans_basic_centers['Cluster_Label'] = kmeans_basic_centers.index

# Merging cluster labels with the original data
data_rfm['Cluster'] = cluster_labels

# Display cluster centers to understand each segment's characteristics
print("Cluster Centers:")
print(kmeans_basic_centers)

In [ ]:
5. ### Outlier Cluster Analysis ###

# Use these boxplots to identify outliers in each cluster
# Look for extreme values that might be skewing your cluster centers
# Consider whether to exclude these outliers and re-run the clustering

# Plotting the boxplots
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Plotting Boxplot of Recency by Cluster
sns.boxplot(x='Cluster', y='recency', data=data_rfm, palette=["#F99E49"])
plt.title('Boxplot of Recency by Cluster', fontsize=18, fontweight='bold')
plt.xlabel('Cluster', fontsize=14, fontweight='bold')
plt.ylabel('Recency', fontsize=14, fontweight='bold')
plt.show()

# Plotting Boxplot of Frequency by Cluster
sns.boxplot(x='Cluster', y='frequency', data=data_rfm, palette=["#F99E49"])
plt.title('Boxplot of Frequency by Cluster', fontsize=18, fontweight='bold')
plt.xlabel('Cluster', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
plt.show()

# Plotting Boxplot of Monetary by Cluster
sns.boxplot(x='Cluster', y='monetary', data=data_rfm, palette=["#F99E49"])
plt.title('Boxplot of Monetary by Cluster', fontsize=18, fontweight='bold')
plt.xlabel('Cluster', fontsize=14, fontweight='bold')
plt.ylabel('Monetary', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
6. ### Rename the Clusterlabels ###

# TODO: Customize cluster names based on your cluster center analysis
# Look at the kmeans_basic_centers output from step 4 to understand each cluster's characteristics
# Example naming conventions:
# - High Value, Medium Value, Low Value
# - Champions, Loyal Customers, At Risk, Lost
# - VIP, Regular, Occasional, Dormant

# Define the mapping from numeric labels to text labels
# The mapping below is an example - adjust based on your actual cluster characteristics
cluster_name_mapping = {
    0: 'A',
    4: 'B',
    3: 'C',
    2: 'D',
    1: 'E'
}  

# Replace numeric labels with text labels in the 'Cluster_Label' column of kmeans_basic_centers
kmeans_basic_centers['Cluster_Label'] = kmeans_basic_centers['Cluster_Label'].replace(cluster_name_mapping)

# Replace numeric labels with text labels in the 'Cluster' column
data_rfm['Cluster'] = data_rfm['Cluster'].replace(cluster_name_mapping)

In [ ]:
7. ### Export Datasets ###

# TODO: Replace "LOCATION/" with your desired export path
# Example: "output/data_rfm.csv" or "C:/Users/YourName/Documents/results/data_rfm.csv"

# Combine Cust_id with Cluster_labels
data_distinct = data[['cust_id']].drop_duplicates()
data_rfm_with_cust_id = pd.merge(data_distinct, data_rfm, on='cust_id')

# Export data_rfm to CSV and Excel
data_rfm_with_cust_id.to_csv('LOCATION/data_rfm.csv', index=False)
data_rfm_with_cust_id.to_excel('LOCATION/data_rfm.xlsx', index=False)

print("Export complete! Your RFM analysis with cluster labels has been saved.")